# Finetuning a LLM with QLoRA

The process in this notebook follows from: https://www.philschmid.de/instruction-tune-llama-2

## Preparing the Dataset

The dataset needed to train Large Language Model are text entries formatted in any way.  This includes any prompts and special tokens that we wish the model to have responses to. 

In this sample lab, we will demonstrate finetuning a Large Language model to respond as an instruction following bot.  The dataset that we will be using is a hand-curated dataset by databricks for instruction following called "dolly-15k".

In [ ]:
from datasets import load_dataset

# Download the dataset from HuggingFace
dataset = load_dataset("databricks/databricks-dolly-15k", split = "train")
dataset

This dataset contains columns for an `instruction`, an optional `context`, and an `response` that we want the bot to respond to.  However, to feed it into the model for finetuning, we need to combine each column so that 1 row corresponds to 1 entry.

In [ ]:
def format_instruction(sample):
    # This function takes a row of the above dataset and returns a single text string

	return f"""### Instruction:
    Use the Input below to create an instruction, which could have been used to generate the input using an LLM.

    ### Input:
    {sample['response']}

    ### Response:
    {sample['instruction']}
    """

## Load the Pretrained Base Model & Tokenizer

Once we have the data that we will be using to fine tune the model, we can select the base model that we would like to fine tune.  The model that we will select is the `Llama-2-7b` base model.  This is a 7b parameter model, quite small in the grand scheme of LLMs, but one that produces good quality results, especially compared to models of similar sizes.

There is quite a bit of knowledge already embedded within the base model, so the fine-tuning we do here will largely just be to change the behavior of the model to act more like a instruction following bot.

In [ ]:
from transformers import TrainingArguments
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Hugging Face Base Model ID
model_id = "NousResearch/Llama-2-7b-hf" # non-gated

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id, 
    quantization_config=bnb_config, 
    use_cache=False, 
    device_map="auto")
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

For memory and computational purposes, we will also be using the `bitsandbytes` library to load and fine tune the model in lower precisions.  More specifically, we will be loading the model weights in 4-bit, compared to a full precision model at 32-bit.  Lowering the precision of the weights, also called ***quantization***,  results in substantial memory and computational savings at the cost of some accuracy.  However, this is common practice now as several studies have shown that the loss in quality of generated responses can be quite minor compared to the benefits.

In [ ]:
# BitsAndBytesConfig int-4 config
# This is for the most memory efficient training
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

## Specify the LoRA Finetuning configuration

There are two main ways to finetune a large language model:

1. Pre-training/Full Finetuning

    In this situation, all of the model weights (all 7b of them) are set to be trainable and tweaked during training.  This can lead to the most dramatic changes in model behavior but is also the most computationally expensive.  When initially training the model, also known as pre-training, this is necessarily done and where you see the extreme computational costs show up (i.e. 500 A100 80GB GPUs trained for 10000 hours, etc...).

2. Parameter Efficient Fine-Tuning

    Parameter efficient finetuning methods are an alternative to full finetuning where, instead of training the parameters of the pre-trained model, a subset of new parameters are trained without touching the base model weights. These new trainable parameters are injected into the model mathematically at different points to change the outcome.  There are a handful of methods that use this approach such as Prompt Tuning, P-Tuning, and Low-Rank Adaptation.  For this lab, we will focus on Low-Rank Adaptation (LoRA).  

    LoRA methods introduce a set of trainable rank-decomposition matrices (update matrices) which can be used to modify the existing weights of the pre-trained model.  The typical location that these matrices are placed are within the attention layers, so they are not exclusive to LLMs.  The size of these update matrices can be controlled by the setting the desired rank of the matrix, with smaller rank corresponding to smaller matrices and thus fewer trainable parameters.   During fine-tuning, only these update matrices are tuned and often times, this makes the total number of trainable parameters a very small fraction of the total number of weights.

Below we define our desired LoRA configuration using the `peft` package.  We will also be using a state-of-the-art method called Quantized LoRA (QLoRA) to also do this training in low precision to save memory.


In [ ]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

# LoRA config for QLoRA
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

# prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

## Finetuning the Model

Now that we have prepared the data, loaded the model in 4-bit, and configured our LoRA finetuning according to our model, we are ready to train the model. 

All of the actual training, including computing gradients, tweaking the optimizer, batching the data, measuring evaluation will be done behind the scenes using the `SFTrainer` wrapper.  This will conduct the finetuning that we want after we pass in the dataset and hyperparameters.  In practice, this is much more efficient and robust than writing our own training code.

In [ ]:
from trl import SFTTrainer

args = TrainingArguments(
    output_dir="llama-7-int4-dolly", 
    num_train_epochs=3,
    per_device_train_batch_size=6,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=2e-4,
    bf16=True,
    tf32=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    disable_tqdm=True # disable tqdm since with packing values are in correct
)

# max sequence length for model and packing of the dataset
max_seq_length = 2048 

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=format_instruction,
    args=args,
)

## Start the Supervisted Finetuning

In [ ]:
trainer.train() # there will not be a progress bar since tqdm is disabled
trainer.save_model()

After the model has finished training, it is ready to be used.  Now, hopefully, when the model sees the prompt that we crafted before, it will know how to respond.